# 모델 학습하기

In [180]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import os

## 하이퍼 파라미터 설정

In [182]:
WINDOW_SIZE = 18               # 3시간 (10분 간격 x 18)
TARGET_OFFSETS = [36, 54, 72]  # 6h, 9h, 12h (10분 간격 기준)
FEATURES = ['LAT', 'LON', 'COG', 'HEADING']
MODEL_PATH = './saved_model/lstm_cluster_1.keras' # 모델명 변경하기 + 각 군집 별로 번호만 수정해주면 됩니당 히히 

## 데이터 불러오기

In [195]:
df = pd.read_csv('cluster1.csv') # 각 cluster 파일명이 cluster1_n.csv 로 구성되어 있음. n의 번호만 바꿔주면 됨 
df = df.sort_values('TIMESTAMP')

OSError: [Errno 22] Invalid argument: 'cluster1.csv'

## 정규화

In [43]:
scaler = MinMaxScaler()
df[FEATURES] = scaler.fit_transform(df[FEATURES])

## 시퀀스 데이터 생성 함수

In [45]:
def create_multistep_sequences(data, window_size, target_offsets):
    X, y = [], []
    for i in range(len(data) - window_size - max(target_offsets)):
        X.append(data[i:i+window_size])
        y.append([data[i+window_size+offset-1] for offset in target_offsets])
    return np.array(X), np.array(y)

## LSTM 모델 정의 함수

In [49]:
def build_model(input_shape, output_dim):
    model = Sequential([
        LSTM(64, input_shape=input_shape),
        Dense(32, activation='relu'),
        Dense(output_dim)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

## 데이터 준비 및 모델 학습

In [51]:
X, y = create_multistep_sequences(df[FEATURES].values, WINDOW_SIZE, TARGET_OFFSETS)
y = y.reshape((y.shape[0], -1))  # flatten: (샘플 수, 타겟시점 x 변수)

In [53]:
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [57]:
model = build_model(X_train.shape[1:], y_train.shape[1])
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10


C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1089/1089 ━━━━━━━━━━━━━━━━━━━━ 18s 13ms/step - loss: 0.0612
Epoch 2/10
1089/1089 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - loss: 0.0402
Epoch 3/10
1089/1089 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - loss: 0.0394
Epoch 4/10
1089/1089 ━━━━━━━━━━━━━━━━━━━━ 15s 13ms/step - loss: 0.0393
Epoch 5/10
1089/1089 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - loss: 0.0390
Epoch 6/10
1089/1089 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - loss: 0.0386
Epoch 7/10
1089/1089 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - loss: 0.0383
Epoch 8/10
1089/1089 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0381
Epoch 9/10
1089/1089 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0377
Epoch 10/10
1089/1089 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - loss: 0.0375


## 모델 저장

In [61]:
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
model.save(MODEL_PATH)
print(f"모델이 저장되었습니다: {MODEL_PATH}")

모델이 저장되었습니다: ./saved_model/lstm_cluster_2.keras


## 예측 결과 및 역변환

In [65]:
y_pred = model.predict(X_test)
y_pred = y_pred.reshape((-1, len(TARGET_OFFSETS), len(FEATURES)))
y_pred_inverse = np.array([
    [scaler.inverse_transform(step.reshape(1, -1))[0] for step in sample]
    for sample in y_pred
])

273/273 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


## 결과 출력

In [76]:
feature_names = ['LAT', 'LON', 'COG', 'HEADING']

for i, offset in enumerate(TARGET_OFFSETS):
    print(f"\n {offset//6}시간 후 예측값 (샘플 5개):")
    for j, sample in enumerate(y_pred_inverse[:5, i, :]):
        print(f"  샘플 {j+1}: ", end="")
        for name, value in zip(feature_names, sample):
            print(f"{name}={value:.4f}  ", end="")
        print() 


 6시간 후 예측값 (샘플 5개):
  샘플 1: LAT=31.3561  LON=122.9937  COG=205.7588  HEADING=229.5551  
  샘플 2: LAT=31.3763  LON=123.0131  COG=209.0277  HEADING=233.5191  
  샘플 3: LAT=31.3715  LON=123.0078  COG=208.8230  HEADING=232.1544  
  샘플 4: LAT=31.3410  LON=122.9394  COG=204.8954  HEADING=225.9318  
  샘플 5: LAT=31.3590  LON=122.9541  COG=206.1213  HEADING=227.2160  

 9시간 후 예측값 (샘플 5개):
  샘플 1: LAT=31.4474  LON=123.1070  COG=201.6828  HEADING=215.5337  
  샘플 2: LAT=31.4780  LON=123.1284  COG=203.7963  HEADING=218.7148  
  샘플 3: LAT=31.4762  LON=123.1029  COG=203.4740  HEADING=219.5950  
  샘플 4: LAT=31.4389  LON=123.0249  COG=200.0087  HEADING=217.4049  
  샘플 5: LAT=31.4495  LON=123.0279  COG=201.7469  HEADING=217.4693  

 12시간 후 예측값 (샘플 5개):
  샘플 1: LAT=31.4186  LON=123.2103  COG=198.4054  HEADING=217.3765  
  샘플 2: LAT=31.4478  LON=123.2438  COG=200.8840  HEADING=220.8494  
  샘플 3: LAT=31.4363  LON=123.2060  COG=201.6132  HEADING=221.4583  
  샘플 4: LAT=31.3895  LON=123.0822  COG=199.2906  HEA

# 새로운 선박 데이터에 대해 예측

In [104]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import folium

## 하이퍼 파라미터 설정

In [106]:
WINDOW_SIZE = 18  # 3시간
TARGET_OFFSETS = [36, 54, 72]
FEATURES = ['LAT', 'LON', 'COG', 'HEADING']
MODEL_PATH = './saved_model/lstm_cluster_2.keras'
feature_names = FEATURES

## 새로운 선박 데이터 불러오기

In [108]:
new_ship_df = pd.read_csv('./dataset/CNSHA/06724e0f-5a08-3aa8-b42c-97acf4f8102d.csv') # 파일명 변경 필요
new_ship_df = new_ship_df.sort_values('TIMESTAMP')

## 정규화 
- 모델 학습했을 때와 동일한 scaler로 맞춰야 함

In [110]:
scaler = MinMaxScaler()
scaler.fit(new_ship_df[FEATURES])
scaled_data = scaler.transform(new_ship_df[FEATURES])

## 시퀀스 함수 생성

In [112]:
def create_input_sequence(data, window_size):
    X = []
    for i in range(len(data) - window_size + 1):
        X.append(data[i:i+window_size])
    return np.array(X)

X_input = create_input_sequence(scaled_data, WINDOW_SIZE)

## 모델 예측

In [114]:
model = load_model(MODEL_PATH)
predicted = model.predict(X_input)
predicted = predicted.reshape((-1, len(TARGET_OFFSETS), len(FEATURES)))

predicted_inverse = np.array([
    [scaler.inverse_transform(step.reshape(1, -1))[0] for step in sample]
    for sample in predicted
])

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step


## 모델 성능 평가 지표
- 개별 선박을 대입했을 때의 예측값과 실제값을 비교해서 평가
- 학습한 LSTM 모델에 특정 선박을 대입했을 때, 이 모델이 얼마나 정확하게 그 선박의 미래 항로를 예측했는가?" 를 평가하는 지표
- 지표 / 의미 / 해석 / 좋은 값
     - MAE (Mean Absolute Error)	/ 평균 절댓값 오차 / 예측값과 실제값의 평균 거리 / 0에 가까울수록 좋음
     - RMSE (Root Mean Squared Error) / 평균 제곱 오차의 루트 / 큰 오차에 더 민감함 / 0에 가까울수록 좋음
     - R² (결정계수) / 설명력 / 예측값이 실제값을 얼마나 잘 설명하는지 / 1에 가까울수록 좋음

In [122]:
num_samples = predicted_inverse.shape[0]
y_true_all = []

for i, offset in enumerate(TARGET_OFFSETS):
    start_idx = WINDOW_SIZE + offset - 1
    end_idx = start_idx + num_samples

    # 선박 데이터 길이를 넘는 경우, 자를 수 없음 → 평가 스킵
    if end_idx > len(new_ship_df):
        print(f"❗ {offset} 오프셋: 데이터 부족으로 평가 생략")
        continue

    y_true = new_ship_df[FEATURES].iloc[start_idx:end_idx].values
    y_true_all.append(y_true)

# 평가 가능한 시점만 진행
if len(y_true_all) == len(TARGET_OFFSETS):  # 모든 시점 평가 가능한 경우
    y_true_all = np.stack(y_true_all, axis=1)  # (샘플 수, 3, 4)

    print("\n 예측 성능 평가 지표:")
    for i, offset in enumerate(TARGET_OFFSETS):
        print(f"\n {offset//6}시간 후 성능 평가:")
        for j, name in enumerate(FEATURES):
            y_true = y_true_all[:, i, j]
            y_pred = predicted_inverse[:, i, j]
            mae = mean_absolute_error(y_true, y_pred)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            r2 = r2_score(y_true, y_pred)
            print(f"  {name}: MAE={mae:.4f}, RMSE={rmse:.4f}, R²={r2:.4f}")
else:
    print("\n 평가 가능한 offset이 부족하여 성능 평가 생략됨.")


 예측 성능 평가 지표:

 6시간 후 성능 평가:
  LAT: MAE=0.3834, RMSE=0.3925, R²=0.7306
  LON: MAE=0.5952, RMSE=0.7337, R²=0.7673
  COG: MAE=10.2357, RMSE=11.7286, R²=0.0742
  HEADING: MAE=13.3528, RMSE=15.2453, R²=-0.0896

 9시간 후 성능 평가:
  LAT: MAE=0.4913, RMSE=0.5130, R²=0.3946
  LON: MAE=0.9120, RMSE=1.0287, R²=0.5316
  COG: MAE=13.9998, RMSE=17.0277, R²=0.0159
  HEADING: MAE=13.2797, RMSE=17.0138, R²=0.3456

 12시간 후 성능 평가:
  LAT: MAE=0.6189, RMSE=0.6454, R²=-0.4969
  LON: MAE=1.3598, RMSE=1.4957, R²=-0.0358
  COG: MAE=15.7010, RMSE=20.8702, R²=0.0376
  HEADING: MAE=15.8036, RMSE=21.2206, R²=0.2046


## Folium 시각화

In [124]:
latlon_actual = new_ship_df[['LAT', 'LON']].values
points_idx = [18, 36, 54, 72]
actual_points = [latlon_actual[i] for i in points_idx if i < len(latlon_actual)]
predicted_latlons = [predicted_inverse[0, i, :2] for i in range(len(TARGET_OFFSETS))]

map_center = actual_points[0] if actual_points else [33.0, 127.0]
m = folium.Map(location=map_center, zoom_start=7)

### 실제 항로

In [126]:
folium.PolyLine(locations=latlon_actual[:points_idx[0]+1], color='blue', weight=3, tooltip='실제 항로').add_to(m)

### 실제 항로의 3시간, 6시간, 9시간, 12시간의 좌표

In [140]:
# 전체 항로를 선으로 표시하려면 이렇게 수정
folium.PolyLine(
    locations=latlon_actual,   # 전체 궤적
    color='blue',
    weight=3,
    tooltip='실제 전체 항로'
).add_to(m)

### 예측 지점 색상별 표시

In [144]:
pred_colors = ['orange', 'red', 'green']
for i, (idx, pred) in enumerate(zip([6, 9, 12], predicted_latlons)):
    folium.CircleMarker(location=pred, radius=6, color=pred_colors[i], fill=True,
                        fill_color=pred_colors[i], tooltip=f'예측 {idx}h').add_to(m)

folium.PolyLine(
    locations=predicted_latlons,
    color='purple',
    weight=3,
    tooltip='예측 항로'
).add_to(m)

In [146]:
display(m)

### 예측값 콘솔 출력 (샘플 1 기준) 
- 소수점 5자리까지 나타낼 수 있게 수정 필요 + csv 파일로 저장할 수 있게 코드 작성 필요
- 각 군집의 번호열에 한줄로 쭉 작성할 수 있게 [6시간 시점: 위도, 경도, COG, HEADING][9시간 지점의 위도, 경도, COG, HEADING] ... 나오게

In [136]:
print(" 예측값 (샘플 1 기준):")
for i, offset in enumerate([6, 9, 12]):
    lat, lon, cog, heading = predicted_inverse[0, i]
    print(f"\n {offset}시간 후 예측")
    print(f"LAT={lat:.4f}, LON={lon:.4f}, COG={cog:.2f}, HEADING={heading:.2f}")

 예측값 (샘플 1 기준):

 6시간 후 예측
LAT=33.8973, LON=127.4547, COG=239.88, HEADING=240.35

 9시간 후 예측
LAT=33.4488, LON=127.5124, COG=251.48, HEADING=236.20

 12시간 후 예측
LAT=33.2169, LON=127.0584, COG=244.01, HEADING=234.80
